In [57]:
import numpy as np
import pandas as pd

In [4]:
# data = pd.read_csv('./Data/CSD Generation (Hourly) - 2020-01 to 2020-06/CSD Generation (Hourly) - 2020-01 to 2020-06.csv')
# renewable_data = data.loc[(data['Fuel Type'] == 'WIND') | (data['Fuel Type'] == 'SOLAR')]
# area_data = renewable_data.loc[renewable_data['Planning Area'] == 52]
# area_data["Fuel Type"].unique()
# area_data["Asset Name"].unique()

## Weather Data

In [5]:
# Read CSVs and drop any unnamed index column
def clean_df(file_path):
    df = pd.read_csv(file_path, encoding='unicode_escape')  # Read CSV file with proper encoding
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]  # Remove any "Unnamed" columns that might be auto-generated
    return df  # Return the cleaned DataFrame

# Load cleaned CSVs (each file contains data for a specific time period)
vauxhaul1 = clean_df('./Data/ACIS/Vauxhall-20200101-20200601.csv')  # Data from Jan 2020 - June 2020
vauxhaul2 = clean_df('./Data/ACIS/Vauxhall-20200701-20201231.csv')  # Data from July 2020 - Dec 2020
vauxhaul3 = clean_df('./Data/ACIS/Vauxhall-20210101-20210630.csv')  # Data from Jan 2021 - June 2021
vauxhaul4 = clean_df('./Data/ACIS/Vauxhall-20210701-20211231.csv')  # Data from July 2021 - Dec 2021

# Concatenate the four dataframes into a single dataframe
vauxhal = pd.concat([vauxhaul1, vauxhaul2, vauxhaul3, vauxhaul4], ignore_index=True)  # `ignore_index=True` resets row indices

# Save the merged dataframe as a new CSV file without index values
vauxhal.to_csv('./Data/ACIS/Vauxhall.csv', index=False)  # `index=False` prevents writing index column to CSV

## Generation Data

In [9]:
df_temp.drop(columns=["timestamp","Date (MST)","Date (MPT)"],inplace=True)

KeyError: "['timestamp', 'Date (MST)', 'Date (MPT)'] not found in axis"

In [72]:
vauxhal.head()

,Station Name,Date (Local Standard Time),Air Temp. Inst. (°C),Air Temp. Inst. Source Flag,Air Temp. Inst. Comment,Humidity Inst. (%),Humidity Inst. Source Flag,Humidity Inst. Comment,Relative Humidity Avg. (%),Relative Humidity Avg. Source Flag,...,Wind Speed 10 m Syno. Comment,Wind Dir. 10 m Syno. (°),Wind Dir. 10 m Syno. Source Flag,Wind Dir. 10 m Syno. Comment,Wind Speed 10 m Avg. (km/h),Wind Speed 10 m Avg. Source Flag,Wind Speed 10 m Avg. Comment,Wind Dir. 10 m Avg. (°),Wind Dir. 10 m Avg. Source Flag,Wind Dir. 10 m Avg. Comment
0,Vauxhall CDA CS,01-January-2020 00:00,2.5,ACTUAL,NaN,74.0,ACTUAL,NaN,76.0,ACTUAL,...,NaN,271.0,ACTUAL,NaN,22.2,ACTUAL,NaN,271.0,ACTUAL,NaN
1,Vauxhall CDA CS,01-January-2020 01:00,2.5,ACTUAL,NaN,75.0,ACTUAL,NaN,75.0,ACTUAL,...,NaN,272.0,ACTUAL,NaN,22.8,ACTUAL,NaN,272.0,ACTUAL,NaN
2,Vauxhall CDA CS,01-January-2020 02:00,2.4,ACTUAL,NaN,77.0,ACTUAL,NaN,77.0,ACTUAL,...,NaN,273.0,ACTUAL,NaN,21.6,ACTUAL,NaN,269.0,ACTUAL,NaN
3,Vauxhall CDA CS,01-January-2020 03:00,2.8,ACTUAL,NaN,78.0,ACTUAL,NaN,78.0,ACTUAL,...,NaN,277.0,ACTUAL,NaN,18.7,ACTUAL,NaN,275.0,ACTUAL,NaN
4,Vauxhall CDA CS,01-January-2020 04:00,0.9,ACTUAL,NaN,83.0,ACTUAL,NaN,81.0,ACTUAL,...,NaN,349.0,ACTUAL,NaN,13.5,ACTUAL,NaN,335.0,ACTUAL,NaN


Checking the number of rows in each DataFrame

In [10]:
# Checking the number of rows in each DataFrame
vauxhaul_Gen_rows = vauxhaul_Gen.shape[0]
vauxhal_rows = vauxhal.shape[0]

# vauxhaul_Gen_rows = len(vauxhaul_Gen)
# vauxhal_rows = len(vauxhal)

print(f"vauxhaul_Gen has {vauxhaul_Gen_rows} rows.")
print(f"vauxhal has {vauxhal_rows} rows.")


vauxhaul_Gen has 16367 rows.
vauxhal has 16846 rows.


Standardize the Date Column, then Merge on Date and Hour

In [11]:
# Convert date columns in GenData (CSD file)
# Convert the 'Date (MST)' column in vauxhaul_Gen to a datetime object
# errors='coerce' ensures that any invalid date formats are converted to NaT (Not a Time)
vauxhaul_Gen['Date'] = pd.to_datetime(vauxhaul_Gen['Date (MST)'], errors='coerce')

# Convert the 'Date (Local Standard Time)' column in vauxhal to a datetime object
vauxhal['Date'] = pd.to_datetime(vauxhal['Date (Local Standard Time)'], errors='coerce')


# Drop the original date columns to avoid confusion and redundancy
# errors='ignore' ensures that the operation does not fail if the column is missing
vauxhaul_Gen.drop(columns=['Date (MST)', 'Date (MPT)'], inplace=True, errors='ignore')
vauxhal.drop(columns=['Date (Local Standard Time)'], inplace=True, errors='ignore')

# Extract the hour from the 'Date' column and store it in a new column called 'Hour'
# This ensures the hour is available for merging datasets
vauxhaul_Gen['Hour'] = vauxhaul_Gen['Date'].dt.hour
vauxhal['Hour'] = vauxhal['Date'].dt.hour

# Merge the two DataFrames on 'Date' and 'Hour' using an inner join
# This ensures that only rows with matching date and hour values are included in the final dataset
merged_df = pd.merge(vauxhal, vauxhaul_Gen, on=['Date', 'Hour'], how='inner')

# Save the merged dataset to a CSV file without the index column
merged_df.to_csv('./Data/Merged/Vauxhall_Merged.csv', index=False)

# Display the first few rows of the merged dataset for verification
print(merged_df.head())

# Store the number of rows in vauxhaul_Gen for reference
vauxhaul_Gen_rows = vauxhaul_Gen.shape[0]

C:\Users\emoni\AppData\Local\Temp\ipykernel_13324\3677057257.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vauxhaul_Gen['Date'] = pd.to_datetime(vauxhaul_Gen['Date (MST)'], errors='coerce')
C:\Users\emoni\AppData\Local\Temp\ipykernel_13324\3677057257.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vauxhaul_Gen.drop(columns=['Date (MST)', 'Date (MPT)'], inplace=True, errors='ignore')
C:\Users\emoni\AppData\Local\Temp\ipykernel_13324\3677057257.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

      Station Name  Air Temp. Inst. (°C) Air Temp. Inst. Source Flag  \
0  Vauxhall CDA CS                 -20.9                      ACTUAL   
1  Vauxhall CDA CS                 -22.2                      ACTUAL   
2  Vauxhall CDA CS                 -22.9                      ACTUAL   
3  Vauxhall CDA CS                 -23.3                      ACTUAL   
4  Vauxhall CDA CS                 -23.3                      ACTUAL   

  Air Temp. Inst. Comment  Humidity Inst. (%) Humidity Inst. Source Flag  \
0                     NaN                82.0                     ACTUAL   
1                     NaN                81.0                     ACTUAL   
2                     NaN                81.0                     ACTUAL   
3                     NaN                81.0                     ACTUAL   
4                     NaN                81.0                     ACTUAL   

  Humidity Inst. Comment  Relative Humidity Avg. (%)  \
0                    NaN                        82.0  

Checking the number of rows in Merged DataFrame

In [12]:
# Assuming you've already merged the files, for example:
merged_df = pd.merge(vauxhaul_Gen, vauxhal, on='Date', how='inner')

# Checking the number of rows in the merged DataFrame
merged_rows = merged_df.shape[0]

# Alternatively, using len()
# merged_rows = len(merged_df)

print(f"The merged file has {merged_rows} rows.")


The merged file has 15669 rows.


In [13]:
print(merged_df.head())  # Check the first few rows of the merged file
print(merged_df.tail())  # Check the last few rows of the merged file

  Asset Short Name     Asset Name Asset Grouping  Volume  Maximum Capability  \
0             VXH1  VXH1 Vauxhall           VXH1     0.0                22.0   
1             VXH1  VXH1 Vauxhall           VXH1     0.0                22.0   
2             VXH1  VXH1 Vauxhall           VXH1     0.0                22.0   
3             VXH1  VXH1 Vauxhall           VXH1     0.0                22.0   
4             VXH1  VXH1 Vauxhall           VXH1     0.0                22.0   

   System Capability Fuel Type Sub Fuel Type  Planning Area Region  ...  \
0               22.0     SOLAR         SOLAR             52  South  ...   
1               22.0     SOLAR         SOLAR             52  South  ...   
2               22.0     SOLAR         SOLAR             52  South  ...   
3               22.0     SOLAR         SOLAR             52  South  ...   
4               22.0     SOLAR         SOLAR             52  South  ...   

  Wind Dir. 10 m Syno. (°)  Wind Dir. 10 m Syno. Source Flag  \
0   

Check for Missing Values

In [14]:
# Check for missing or invalid date values
print(merged_df.isna().sum())


Asset Short Name                          0
Asset Name                                0
Asset Grouping                            0
Volume                                    0
Maximum Capability                        0
System Capability                         0
Fuel Type                                 0
Sub Fuel Type                             0
Planning Area                             0
Region                                    0
Date                                      0
Hour_x                                    0
Station Name                              0
Air Temp. Inst. (°C)                      0
Air Temp. Inst. Source Flag               0
Air Temp. Inst. Comment               13005
Humidity Inst. (%)                        0
Humidity Inst. Source Flag                0
Humidity Inst. Comment                13006
Relative Humidity Avg. (%)                0
Relative Humidity Avg. Source Flag        0
Relative Humidity Avg. Comment        13005
Incoming Solar Rad. (W/m2)      

Handling Missing Data: Drop Columns with Too Many Missing Values (70%)

In [15]:
# Set the threshold for missing values (70% missing values)
threshold = 0.7

# Calculate the percentage of missing values for each column
missing_percentage = merged_df.isna().mean()

# Identify columns that have more than the threshold percentage of missing values
columns_to_drop = missing_percentage[missing_percentage > threshold].index

# Drop those columns
merged_df.drop(columns=columns_to_drop, axis=1, inplace=True)

# Check the result
print(merged_df.isna().sum())

Asset Short Name                         0
Asset Name                               0
Asset Grouping                           0
Volume                                   0
Maximum Capability                       0
System Capability                        0
Fuel Type                                0
Sub Fuel Type                            0
Planning Area                            0
Region                                   0
Date                                     0
Hour_x                                   0
Station Name                             0
Air Temp. Inst. (°C)                     0
Air Temp. Inst. Source Flag              0
Humidity Inst. (%)                       0
Humidity Inst. Source Flag               0
Relative Humidity Avg. (%)               0
Relative Humidity Avg. Source Flag       0
Incoming Solar Rad. (W/m2)               0
Incoming Solar Rad. Source Flag          0
Incoming Solar Rad. Comment              0
Precip. (mm)                             0
Precip. Sou

Check for Duplicates

In [16]:
print(merged_df.duplicated().sum())

0


Convert Categorical Variables: Nominal (Categorical) Data (No Order or Ranking) so one-hot encoding will create a new binary column for each category

In [17]:
# Convert categorical variables into dummy/indicator variables (one-hot encoding)
# drop_first=True removes the first category to avoid multicollinearity in regression models
merged_df = pd.get_dummies(merged_df, drop_first=True)

Convert Date Columns

In [18]:
# Extracting additional time-related features from the 'Date' column

# Extract the year from the Date column
merged_df['Year'] = merged_df['Date'].dt.year

# Extract the month from the Date column
merged_df['Month'] = merged_df['Date'].dt.month

# Extract the day from the Date column
merged_df['Day'] = merged_df['Date'].dt.day

# Extract the hour from the Date column (redundant if already created earlier)
merged_df['Hour'] = merged_df['Date'].dt.hour

# Extract the weekday (0 = Monday, 6 = Sunday)
merged_df['Weekday'] = merged_df['Date'].dt.weekday

Drop the original Date column

In [19]:
# Drop the 'Date' column since we have already extracted relevant time-related features
merged_df.drop('Date', axis=1, inplace=True)

In [20]:
print(merged_df.columns)
print(len(merged_df.columns))

Index(['Volume', 'Maximum Capability', 'System Capability', 'Planning Area',
       'Hour_x', 'Air Temp. Inst. (°C)', 'Humidity Inst. (%)',
       'Relative Humidity Avg. (%)', 'Incoming Solar Rad. (W/m2)',
       'Precip. (mm)',
       ...
       'Wind Speed 2 m Avg. Comment_wind profile WSA=9.900',
       'Wind Speed 10 m Syno. Source Flag_ESTIMATED',
       'Wind Dir. 10 m Syno. Source Flag_MISSING',
       'Wind Speed 10 m Avg. Source Flag_ESTIMATED',
       'Wind Dir. 10 m Avg. Source Flag_MISSING', 'Year', 'Month', 'Day',
       'Hour', 'Weekday'],
      dtype='object', length=3871)
3871


Split Data into Features and Target

In [21]:
# Define the target variable: 'Volume' (the power generated)
y = merged_df['Volume']  # Power generation output that we want to predict

# Define the features: all columns except 'Volume'
X = merged_df.drop('Volume', axis=1)  # Drop the 'Volume' column to use all others as features


Split the Data into Training and Test Sets

In [22]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
# X contains the features (independent variables), y contains the target (dependent variable)
# The test size is set to 20%, meaning 80% of the data will be used for training, and 20% will be used for testing.
# random_state ensures the split is reproducible, so you get the same split every time you run the code.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Train the Random Forest Model

In [23]:
from sklearn.ensemble import RandomForestRegressor
# Initialize the Random Forest model
# rf = RandomForestRegressor(random_state=42)
# Reduce complexity by limiting max depth and increasing min samples
rf = RandomForestRegressor(
    n_estimators=100,  # You can try fewer estimators to prevent overfitting
    max_depth=10,  # Limit depth to prevent deep trees
    min_samples_split=10,  # Increase the minimum number of samples required to split a node
    min_samples_leaf=5,  # Increase the minimum number of samples required at a leaf node
    max_features='sqrt',  # Use a subset of features at each split to prevent overfitting
    random_state=42
)

# Train the model on the training dataset (X_train, y_train)
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, max_features='sqrt', min_samples_leaf=5,
                      min_samples_split=10, random_state=42)

Evaluate the Model

In [24]:
from sklearn.metrics import mean_absolute_error, mean_squared_log_error, r2_score

# Predict the target values using the trained model on both training and test sets
y_train_pred = rf.predict(X_train)  # Predictions on the training set
y_test_pred = rf.predict(X_test)    # Predictions on the test set

# Calculate the Mean Absolute Error (MAE) for both training and test sets
# MAE measures the average absolute difference between predicted and actual values.
train_mae = mean_absolute_error(y_train, y_train_pred)
valid_mae = mean_absolute_error(y_test, y_test_pred)

# Calculate the Root Mean Squared Logarithmic Error (RMSLE) for both training and test sets
# RMSLE is often used for regression tasks when predictions can have a wide range.
# It penalizes underestimations more than overestimations.
train_rmsle = np.sqrt(mean_squared_log_error(y_train, y_train_pred))
valid_rmsle = np.sqrt(mean_squared_log_error(y_test, y_test_pred))

# Calculate the R-squared (R^2) score for both training and test sets
# R^2 measures how well the model explains the variance of the target variable.
# Higher values closer to 1 indicate better fit.
train_r2 = r2_score(y_train, y_train_pred)
valid_r2 = r2_score(y_test, y_test_pred)

# Create a dictionary to store and organize all the calculated results
results = {
    'Training MAE': train_mae,  # MAE for training set
    'Valid MAE': valid_mae,     # MAE for validation/test set
    'Training RMSLE': np.float64(train_rmsle),  # RMSLE for training set
    'Valid RMSLE': np.float64(valid_rmsle),    # RMSLE for validation/test set
    'Training R^2': train_r2,  # R^2 for training set
    'Valid R^2': valid_r2     # R^2 for validation/test set
}

# Print the results dictionary containing all the metrics
print(results)

{'Training MAE': 4.05236436573621, 'Valid MAE': 3.962469137816137, 'Training RMSLE': np.float64(1.126544570378072), 'Valid RMSLE': np.float64(1.1194551169094493), 'Training R^2': 0.3686902052341898, 'Valid R^2': 0.3767715995119705}


The performance (both MAE and R²) is relatively low, suggesting that the model might not be capturing the underlying patterns in the data well.

Feature Importance

In [25]:
# Get feature importances from the trained RandomForest model
feature_importances = rf.feature_importances_  # Importance of each feature in the model's prediction

# Get the names of the features (columns)
feature_names = X.columns  # Feature names from the training data

# Create a DataFrame to organize feature names and their corresponding importances for easy viewing
importance_df = pd.DataFrame({
    'Feature': feature_names,         # Feature names
    'Importance': feature_importances  # Corresponding importance values
})

# Sort the DataFrame by the 'Importance' column in descending order to see the most important features first
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Print the DataFrame containing feature names and their importance values
print(importance_df)

                                                Feature  Importance
7                            Incoming Solar Rad. (W/m2)    0.122119
3                                                Hour_x    0.112383
5                                    Humidity Inst. (%)    0.093431
14                                               Hour_y    0.081852
3868                                               Hour    0.080604
...                                                 ...         ...
25    Incoming Solar Rad. Comment_IDW I.R.: power=2,...    0.000000
26    Incoming Solar Rad. Comment_IDW I.R.: power=2,...    0.000000
27    Incoming Solar Rad. Comment_IDW I.R.: power=2,...    0.000000
193   Precip. Comment_benchmark greater than reading...    0.000000
3853  Wind Speed 2 m Avg. Comment_wind profile WSA=9...    0.000000

[3870 rows x 2 columns]


Use Feature Importance to reduce number of features

In [26]:
# Set a threshold for feature importance (e.g., 1% of the maximum importance)
threshold = 0.01 * importance_df['Importance'].max()

# Filter out features with importance less than the threshold
important_features = importance_df[importance_df['Importance'] >= threshold]

# Get the list of important features
features_to_keep = important_features['Feature'].tolist()

# Get the list of removed features (those with importance less than the threshold)
features_removed = importance_df[importance_df['Importance'] < threshold]['Feature'].tolist()

# Filter X to keep only important features
X_filtered = X[features_to_keep]

# Print the features kept and removed
print(f"Features kept: {features_to_keep}")
print(f"Features removed: {features_removed}")


Features kept: ['Incoming Solar Rad. (W/m2)', 'Hour_x', 'Humidity Inst. (%)', 'Hour_y', 'Hour', 'Air Temp. Inst. (°C)', 'Relative Humidity Avg. (%)', 'Precip. Source Flag_ESTIMATED', 'Month', 'Relative Humidity Avg. Source Flag_ESTIMATED', 'Incoming Solar Rad. Comment_IDW I.R.: power=2, radius=60.0, stations=8, nearest=25.5, farthest=59.1, average=37.3', 'Year', 'Incoming Solar Rad. Comment_IDW I.R.: power=2, radius=60.0, stations=8, nearest=25.5, farthest=59.1, average=40.3', 'Wind Dir. 10 m Avg. (°)', 'Incoming Solar Rad. Comment_IDW I.R.: power=2, radius=60.0, stations=8, nearest=25.5, farthest=58.5, average=37.3', 'Humidity Inst. Source Flag_ESTIMATED', 'Wind Speed 2 m Avg. Comment_IDW I.R.: power=2, radius=60.0, stations=8, nearest=25.5, farthest=58.5, average=37.3', 'Wind Speed 10 m Avg. Source Flag_ESTIMATED', 'Wind Dir. 10 m Avg. Source Flag_MISSING', 'Precip. Comment_benchmark greater than reading: reading=428.5, benchmark=708.2, seton=2020-08-13T04:00;IDW I.R.: power=2, radiu

Cross-validation

In [27]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
import numpy as np


# Perform 5-fold cross-validation
cv_scores = cross_val_score(rf, X, y, cv=5, scoring='neg_mean_absolute_error')

# Convert negative MAE to positive
cv_scores = -cv_scores

# Print out the cross-validation scores
print("Cross-validation MAE scores:", cv_scores)
print("Average Cross-validation MAE:", np.mean(cv_scores))


Cross-validation MAE scores: [4.62014414 3.98637669 3.19162674 5.73046032 3.85775053]
Average Cross-validation MAE: 4.277271683917489


Underfitting: Given that your R² values are low, it seems that your model is underfitting. It is not capturing enough of the variance in the data. This indicates that either the model is too simple or the features used in the model are insufficient or not well-prepared for the task at hand.

RandomizedSearchCV to find best parameters for Hyperparameter Tuning

In [29]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_dist = {
    'n_estimators': randint(100, 1000),
    'max_depth': [10, 20, 30, None],
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': ['auto', 'sqrt', 'log2']
}

random_search = RandomizedSearchCV(estimator=RandomForestRegressor(), param_distributions=param_dist, n_iter=100, cv=5, n_jobs=-1, verbose=2, random_state=42)
random_search.fit(X_train, y_train)

# Get the best parameters
print("Best parameters:", random_search.best_params_)

# Get the best model
best_model = random_search.best_estimator_


Fitting 5 folds for each of 100 candidates, totalling 500 fits


c:\Users\emoni\anaconda3\envs\enel678\lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
190 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
140 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\emoni\anaconda3\envs\enel678\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\emoni\anaconda3\envs\enel678\lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
  File "c:\Users\emoni\anaconda3\envs\enel678\lib\site-packages\sklearn\base.py", line 436, in _validate_params
    validate_parameter_constrain

Best parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 17, 'n_estimators': 602}


Retrain the model with the best parameters

In [31]:
from sklearn.ensemble import RandomForestRegressor

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)


# Best hyperparameters from RandomizedSearchCV
best_params = {
    'max_depth': None,
    'max_features': 'sqrt',
    'min_samples_leaf': 1,
    'min_samples_split': 17,
    'n_estimators': 602
}

# Create a new RandomForestRegressor with the best parameters
rf_best = RandomForestRegressor(**best_params)

# Fit the model on the training data
rf_best.fit(X_train, y_train)

# Evaluate the model on the training set
train_predictions = rf_best.predict(X_train)
train_mae = mean_absolute_error(y_train, train_predictions)
train_r2 = r2_score(y_train, train_predictions)

# Evaluate the model on the validation set
valid_predictions = rf_best.predict(X_valid)
valid_mae = mean_absolute_error(y_valid, valid_predictions)
valid_r2 = r2_score(y_valid, valid_predictions)

# Print the performance metrics
print(f"Training MAE: {train_mae}")
print(f"Validation MAE: {valid_mae}")
print(f"Training R^2: {train_r2}")
print(f"Validation R^2: {valid_r2}")


Training MAE: 1.4089817846642048
Validation MAE: 1.822427308806352
Training R^2: 0.8865811150826481
Validation R^2: 0.8092859717725176


Good Model Fit: Both the Training MAE and Validation MAE are relatively low, which indicates that the model's predictions are close to the actual values.

R² Value: The Training R² value of 0.89 suggests that the model is explaining 89% of the variance in the training data, which is excellent. The Validation R² of 0.81 still indicates good performance but shows a slight drop compared to the training set.

Overfitting: The model is overfitting slightly, as indicated by the higher performance on the training data (R² is higher and MAE is lower) compared to the validation data. This is a common scenario when the model has learned the specifics of the training data well but might not generalize as effectively to unseen data.

In [32]:
# Perform 5-fold cross-validation
cv_scores = cross_val_score(rf_best, X, y, cv=5, scoring='neg_mean_absolute_error')

# Convert negative MAE to positive
cv_scores = -cv_scores

# Print out the cross-validation scores
print("Cross-validation MAE scores:", cv_scores)
print("Average Cross-validation MAE:", np.mean(cv_scores))

Cross-validation MAE scores: [3.87146483 2.45755446 1.8044686  3.63613458 2.34690337]
Average Cross-validation MAE: 2.8233051692038833


Good generalization: Despite the variation, the average MAE of 2.82 indicates that the model is performing fairly well overall.

Model Stability: The variation between different folds suggests that there may be some instability in how the model generalizes to different parts of the dataset. It could be useful to explore whether certain features or data points are causing this inconsistency.

In [20]:
# # Splitting train and validation data
# df_val = df_temp[df_temp.year == 2020]
# df_train = df_temp[df_temp.year != 2020]

# # Checking data sizes
# print(len(df_val), len(df_train))

# # Splitting into features (X) and target (y)
# X_train, y_train = df_train.drop(columns=["Volume"]), df_train["Volume"]
# X_valid, y_valid = df_val.drop(columns=["Volume"]), df_val["Volume"]

# # Checking shapes
# X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

In [85]:
from sklearn.model_selection import TimeSeriesSplit

# Assuming 'df_temp' is already sorted by 'year' (or by time column)
# Splitting into features (X) and target (y)
X = df_temp.drop(columns=["Volume"])
y = df_temp["Volume"]

# Checking the shape before splitting
print(X.shape, y.shape)

# Initialize TimeSeriesSplit with the desired number of splits
tscv = TimeSeriesSplit(n_splits=5)

# Loop through the splits and get training and validation data
for train_index, valid_index in tscv.split(X):
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    # Check shapes of train and validation sets for each split
    print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape)


(376058, 62) (376058,)
(62678, 62) (62678,) (62676, 62) (62676,)
(125354, 62) (125354,) (62676, 62) (62676,)
(188030, 62) (188030,) (62676, 62) (62676,)
(250706, 62) (250706,) (62676, 62) (62676,)
(313382, 62) (313382,) (62676, 62) (62676,)


In [86]:
%%time
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

# Different RandomForestClassifier hyperparameters
rf_grid = {
    "n_estimators": np.arange(10, 50, 10),  # Reduce complexity
    "max_depth": [5, 10, 15],  # Limit depth
    "min_samples_split": np.arange(5, 20, 5),  # Encourage splits on more data
    "min_samples_leaf": np.arange(5, 20, 5),  # Prevent overfitting on small samples
    "max_features": ["sqrt", 0.5],
    "max_samples": [None]  # Allow dynamic sampling
}

rs_model = RandomizedSearchCV(RandomForestRegressor(),
                              param_distributions=rf_grid,
                              n_iter=20,
                              cv=5,
                              verbose=True)

rs_model.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
CPU times: total: 8min 45s
Wall time: 8min 52s


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=20,
                   param_distributions={'max_depth': [5, 10, 15],
                                        'max_features': ['sqrt', 0.5],
                                        'max_samples': [None],
                                        'min_samples_leaf': array([ 5, 10, 15]),
                                        'min_samples_split': array([ 5, 10, 15]),
                                        'n_estimators': array([10, 20, 30, 40])},
                   verbose=True)

In [87]:
rs_model.best_params_

{'n_estimators': np.int64(20),
 'min_samples_split': np.int64(10),
 'min_samples_leaf': np.int64(5),
 'max_samples': None,
 'max_features': 0.5,
 'max_depth': 5}

In [88]:
%%time
# Most ideal hyperparameters
ideal_model = RandomForestRegressor(n_estimators=90,
                                    min_samples_leaf=1,
                                    min_samples_split=14,
                                    max_features=0.5,
                                    n_jobs=-1,
                                    max_samples=None)
ideal_model.fit(X_train, y_train)

CPU times: total: 1min 21s
Wall time: 13.4 s


RandomForestRegressor(max_features=0.5, min_samples_split=14, n_estimators=90,
                      n_jobs=-1)

In [89]:
def rmsle(y_test, y_preds):
    return np.sqrt(mean_squared_log_error(y_test, y_preds))
# Create function to evaluate our model
from sklearn.metrics import mean_squared_log_error, mean_absolute_error
def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_valid)
    scores = {"Training MAE": mean_absolute_error(y_train, train_preds),
              "Valid MAE": mean_absolute_error(y_valid, val_preds),
              "Training RMSLE": rmsle(y_train, train_preds),
              "Valid RMSLE": rmsle(y_valid, val_preds),
              "Training R^2": model.score(X_train, y_train),
              "Valid R^2": model.score(X_valid, y_valid)}
    return scores

In [90]:
show_scores(ideal_model)

{'Training MAE': 4.317008117571929,
 'Valid MAE': 4.807101240960573,
 'Training RMSLE': np.float64(1.1680304665714298),
 'Valid RMSLE': np.float64(1.4806566227149753),
 'Training R^2': 0.2811729630308666,
 'Valid R^2': -0.18345195058558028}